In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Import mlcompute module to use the optional set_mlc_device API for device selection with ML Compute.
from tensorflow.python.compiler.mlcompute import mlcompute
from konlpy.tag import Mecab
import pickle
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

ModuleNotFoundError: No module named 'ko_restoration'

In [2]:
pd.set_option('display.unicode.east_asian_width', True)


In [3]:
raw_df = pd.read_csv('/Users/san/work/python/Deep_Learning/LSTM_DNN_PJT/data/raw_data.csv', index_col=0)
raw_df.head(10)

,Title,Medium_category,Small_category,Introduction,cnt
1,-10KG 밀가루 단식,가정 살림,요리,내 몸 리셋 다이어트 10kg 밀가루 단식 저자의 100일 밀가루 단식 다이어...,1
2,디디미니의 초간단 인생맛 고단백 저탄수화물 다이어트 레시피 ...,가정 살림,요리,맛있고 배부른데 살까지 빠지는 다이어트 레시피 끝판왕 101가지 2018년...,1
3,맛있게 쓴 옥주부 레시피 100,가정 살림,요리,옥주부가 매일 자신의 인스타그램에 밥상 메뉴를 올리는 이유는 뭐 먹고 사는지 ...,1
4,세상 쉽고 맛있는 튼이 이유식,가정 살림,요리,쌀가루 큐브 밥솥 칸막이를 활용한 최초의 이유식 책 엄마의 정성과 사랑...,1
5,맛있게 살 빠지는 고단백 저탄수화물 다이어트 레시피...,가정 살림,요리,다이어트 할 때도 맛있게 먹어야 성공한다 미니는 22kg 엄마는 17kg을 ...,1
6,임지호의 밥 땅으로부터,가정 살림,요리,면역력이 시대의 슬로건이 된 이때 모두에게 추천하는 임지호의 들풀밥상 ...,1
7,유아식 레시피북,가정 살림,요리,아이가 잘 안 먹는 것은 엄마 잘못이 아니에요 돌부터 요리를 시작하는 ...,1
8,와인이 있는 100가지 장면,가정 살림,요리,라라랜드 에서 미아가 꿈속에서 마시는 와인은 뭘까 아이언맨 의 토니 스타...,1
9,백종원이 추천하는 집밥 메뉴 애장판,가정 살림,요리,국민 요리책 백종원이 추천하는 집밥 메뉴 시리즈 합본 한정판 1 4탄 백...,1
10,[예스리커버] 매직 레시피,가정 살림,요리,난생처음 요리하는 당신이라도 전문점 맛 그대로 YouTube 요리 ...,1


In [4]:
print('initial:', len(raw_df))

initial: 41170


## data 공백 처리

In [5]:
for i in range(len(raw_df)):
    for j in range(5, 1, -1):
        raw_df.iloc[i,3] = raw_df.iloc[i,3].replace(' '*j,  ' ')

In [6]:
print('after gap-healing:', len(raw_df))

after gap-healing: 41170


## data 중복 제거

In [7]:
# null값 확인
# raw_df['Introduction'].isnull().values.any()

In [8]:
# 중복된 data 개수 확인
sum_dup = raw_df.Introduction.duplicated().sum()
sum_dup

184

In [9]:
# 중복된 data 제거(row)
df = raw_df.drop_duplicates(subset=['Introduction'])
sum_dup = df.Introduction.duplicated().sum()
sum_dup

0

In [10]:
print('after dropna', len(df))

after dropna 40986


In [11]:
# 새로운 index 할당
df.reset_index(drop=True, inplace=True) # drop=True : 기존 index를 제거
df

,Title,Medium_category,Small_category,Introduction,cnt
0,-10KG 밀가루 단식,가정 살림,요리,내 몸 리셋 다이어트 10kg 밀가루 단식 저자의 100일 밀가루 단식 다이어트 ...,1
1,디디미니의 초간단 인생맛 고단백 저탄수화물 다이어트 레시피 ...,가정 살림,요리,맛있고 배부른데 살까지 빠지는 다이어트 레시피 끝판왕 101가지 2018년 201...,1
2,맛있게 쓴 옥주부 레시피 100,가정 살림,요리,옥주부가 매일 자신의 인스타그램에 밥상 메뉴를 올리는 이유는 뭐 먹고 사는지 자랑...,1
3,세상 쉽고 맛있는 튼이 이유식,가정 살림,요리,쌀가루 큐브 밥솥 칸막이를 활용한 최초의 이유식 책 엄마의 정성과 사랑이 듬뿍 담...,1
4,맛있게 살 빠지는 고단백 저탄수화물 다이어트 레시피...,가정 살림,요리,다이어트 할 때도 맛있게 먹어야 성공한다 미니는 22kg 엄마는 17kg을 감량하...,1
...,...,...,...,...,...
40981,한국인의 초보 사주팔자,인문,명리/주역/풍수,인생 쉽지 않다 생각한 대로 흘러가지 않기 때문이다 지성과 이성으로 내린 선택과 ...,1
40982,다르게 살고 싶다,인문,명리/주역/풍수,나는 어떤 사람인가 타인의 욕망을 좇는 삶에서 자기 주도적인 삶으로 이 책은 스펙...,1
40983,주역과 만나다 하,인문,명리/주역/풍수,주역 은 한마디로 변화 의 책이다 변화는 밤하늘을 화려하게 수놓는 일월성신의 규칙...,1
40984,주역과 만나다 중,인문,명리/주역/풍수,주역 은 하늘과 땅 해와 달 바람과 우레 뫼와 연못이라는 여덟 가지 형상을 우주의...,1


In [12]:
Medium_ctg = df['cnt'].groupby(df['Medium_category'])  # 12개의 category
Medium_ctg.sum() 

Medium_category
가정 살림           2494
건강 취미           3156
경제 경영           3607
국어 외국어 사전    2496
만화/라이트노벨     5694
사회 정치           2858
소설/시/희곡        4774
수험서 자격증       3716
어린이              3806
유아                3084
인문                2720
청소년              2581
Name: cnt, dtype: int64

## Book Introduction Preprocessing

## data를 X, Y로 분할

In [13]:
X = df['Introduction']
Y = df['Medium_category']

## Y(label) 처리

In [14]:
# Y값 label Encoding
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)
print(labeled_Y)

['가정 살림' '건강 취미' '경제 경영' '국어 외국어 사전' '만화/라이트노벨' '사회 정치' '소설/시/희곡'
 '수험서 자격증' '어린이' '유아' '인문' '청소년']
[ 0  0  0 ... 10 10 10]


In [15]:
# encoding mapping 정보를 저장
with open('/Users/san/work/python/Deep_Learning/LSTM_DNN_PJT/data/category_encoder_12.pickle', 'wb') as f:
  pickle.dump(encoder, f)

In [16]:
# label을 onehot encoding으로 변환
onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


## X(data) 처리

### 형태소 분석

In [20]:
tokenizer = Mecab()

In [30]:
text = '형태소 분석 이란 형태소를 비롯하여,\
어근, 접두사/접미사, 품사(POS, part-of-speech) 등\
다양한 언어적 속성의 구조를 파악하는 것입니다'

pos_test = tokenizer.pos(text)
morphs_test = tokenizer.morphs(text)
noun_test = tokenizer.nouns(text)
pos_1_test = []
for temp in pos_test:
    pos_1_test.append(temp[0])
    
print('#'*15, 'pos_test', '#'*15, '\n', pos_test)
print('#'*15, 'pos_1_test', '#'*15, '\n', pos_1_test)
print('#'*15, 'noun_test', '#'*15, '\n', noun_test)
print('#'*15, 'morphs_test', '#'*15, '\n', morphs_test)

############### pos_test ############### 
 [('형태소', 'NNG'), ('분석', 'NNG'), ('이', 'VCP'), ('란', 'ETM'), ('형태소', 'NNG'), ('를', 'JKO'), ('비롯', 'XR'), ('하', 'XSA'), ('여', 'EC'), (',', 'SC'), ('어근', 'NNG'), (',', 'SC'), ('접두사', 'NNG'), ('/', 'SC'), ('접미사', 'NNG'), (',', 'SC'), ('품사', 'NNG'), ('(', 'SSO'), ('POS', 'SL'), (',', 'SC'), ('part', 'SL'), ('-', 'SY'), ('of', 'SL'), ('-', 'SY'), ('speech', 'SL'), (')', 'SSC'), ('등', 'NNB'), ('다양', 'NNG'), ('한', 'XSA+ETM'), ('언어', 'NNG'), ('적', 'XSN'), ('속성', 'NNG'), ('의', 'JKG'), ('구조', 'NNG'), ('를', 'JKO'), ('파악', 'NNG'), ('하', 'XSV'), ('는', 'ETM'), ('것', 'NNB'), ('입니다', 'VCP+EC')]
############### pos_1_test ############### 
 ['형태소', '분석', '이', '란', '형태소', '를', '비롯', '하', '여', ',', '어근', ',', '접두사', '/', '접미사', ',', '품사', '(', 'POS', ',', 'part', '-', 'of', '-', 'speech', ')', '등', '다양', '한', '언어', '적', '속성', '의', '구조', '를', '파악', '하', '는', '것', '입니다']
############### noun_test ############### 
 ['형태소', '분석', '형태소', '어근', '접두사', '접미사', '품사', '등', 

In [ ]:
tokenizer = Mecab()
print('형태소 분석')
for i in range(len(X)):
    X[i] = tokenizer.noun(X[i])
    if (i % 250 == 0) and (i>1):
        print('.', end='')
    if i % 5000 == 0:
        print('{} / {}'.format(i, len(X)))
print(X)

In [ ]:
print(X.shape)
print(len(X))

In [ ]:
x_len_shape = []
for x in X:
    x_len_shape.append(len(x))
x_len_shape

In [ ]:
test = pd

In [ ]:
for i in range(len(X)):
    if len(X[i]) >=2000:
        X[i] = X[i][:2000]

In [ ]:
!pwd

In [ ]:
# 형태소 분석 된 X data 저장
X.to_csv('../data/cat_12_morphs_X_MeCab_len_3000.csv')

### 불용어 제거
- 영어, 숫자도 제거할 것인지?

In [ ]:
# 불용어 제거
kor_stopwords = pd.read_csv('../data/stopwords.csv')
nltk.download('stopwords')
eng_stopwords = set(stopwords.words('english'))
stopword = list(kor_stopwords['stopword']) + list(eng_stopwords)
# 불용어 제거 후 형태소로 이루어진 문장으로 재조합
for i in range(len(X)) :
    result = []
    for j in range(len(X[i])):
        if len(X[i][j]) > 1:  # 길이가 한 글자인 것은 지움
            if X[i][j] not in stopword:
                result.append(X[i][j])
#             elif X[i][j] not in list(eng_stopwords):
#                 result.append(X[i][j])
        
    X[i] = ' '.join(result)
    if (i % 250 == 0) and (i>1):
        print('.', end='')
    if i % 5000 == 0:
        print('{} / {}'.format(i, len(X)))
print(X)

In [ ]:
# 불용어 제거된 X data 저장
X.to_csv('../data/cat_12_morphs_X_MeCab_len_3000_stopwords_removed.csv')

### 토크나이징

In [ ]:
# tokenizing : 각 형태소에 숫자 label값을 배정
token = Tokenizer()
token.fit_on_texts(X)  # 형태소에 어떤 숫자를 배정할 것인지
tokened_X = token.texts_to_sequences(X)  # 토큰에 저장된 label을 바탕으로 문장(X)을 변환
print(tokened_X[0])

In [ ]:
# token 저장
# tokened_X.to_csv('./data/tokened_X_3.csv')

In [ ]:
# 데이터 형태 그대로 저장
with open('../data/cat_12_MeCab_3000_book_token.pickle', 'wb') as f:
  pickle.dump(token, f)

## data 확인

In [ ]:
# 형태소 개수 확인
wordsize = len(token.word_index) + 1
# print('word index : ', token.word_index)
print('wordsize is : ', wordsize)  # index 0를 padding 으로 추가 예정


In [ ]:
## tokend_X의 아웃라이어 확인하기!!!!!!!
tokened_len = []
for i in range(len(tokened_X)):
    tokened_len.append(len(tokened_X[i]))

mean = np.mean(tokened_len)
max = int(np.trunc(mean))
print(max)

In [ ]:
# # 1. 가장 긴 문장의 길이 확인
# max = 0
# for i in range(len(tokened_X)):
#   if max < len(tokened_X[i]):
#       max = len(tokened_X[i])
      
#       print('max is : ', max) 

In [ ]:
# padding
X_pad = pad_sequences(tokened_X, max) # 앞쪽을 0으로 채움

# X_pad.to_csv('./data/padded_X_4.csv')

print(X_pad[:10])

## Train, Test set split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_pad, onehot_Y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

## Train, Test set 저장

In [ ]:
xy = X_train, X_test, Y_train, Y_test
np.save('../data/book_data_max_{}_wordsize_{}'.format(max, wordsize), xy)